In [1]:
import ee
import geemap

In [2]:
import ee
ee.Authenticate()
ee.Initialize(project='glodal-projects')

In [3]:
# !conda install -c conda-forge ipyleaflet

In [4]:
Map = geemap.Map(center=[28.63,82.27], zoom=8)
Map

Map(center=[28.63, 82.27], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

In [5]:
import ee
import geopandas as gpd


# Load the shapefile as a GeoDataFrame and convert to EPSG:4326
kailali_gdf = gpd.read_file('./Output/kailali_poly_filtered.shp')
kailali_gdf = kailali_gdf.to_crs(epsg=4326)


kailali_fc = geemap.geopandas_to_ee(kailali_gdf)

# Get the extent (bounding box) of the GeoDataFrame
bounds = kailali_gdf.total_bounds  # returns [minx, miny, maxx, maxy]

# Create an ee.Geometry object using the bounding box
aoi = ee.Geometry.Rectangle([bounds[0], bounds[1], bounds[2], bounds[3]])

# Define the time range
start_date = '2024-06-01'
end_date = '2024-08-01'

# Function to mask clouds based on the Sentinel-2 cloud probability band
def mask_s2_clouds(image):
    cloud_prob = image.select('MSK_CLDPRB')
    is_not_cloudy = cloud_prob.lt(20)  # Adjust the threshold as needed
    return image.updateMask(is_not_cloudy).divide(10000)

# Fetch the Sentinel-2 image collection and apply cloud masking
composite = (
    ee.ImageCollection('COPERNICUS/S2_SR')
    .filterBounds(aoi)
    .filterDate(start_date, end_date)
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
    .map(mask_s2_clouds)
    .median()
)

# Calculate NDVI
ndvi = composite.normalizedDifference(['B8', 'B4']).rename('NDVI')

# Define visualization parameters for NDVI
ndvi_vis_params = {
    'min': -1,
    'max': 1,
    'palette': ['blue', 'white', 'green']
}

# Define visualization parameters
vis_params = {
    'min': 0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],  # RGB
}

# Create a map object
Map = geemap.Map()

# Center the map on the AOI and add the AOI layer
Map.centerObject(aoi, 10)
Map.add_gdf(kailali_gdf, layer_name="Kailali Samples")
# Add the cloud-free composite to the map
Map.addLayer(composite, vis_params, 'Kailali composite')
# Add the NDVI layer to the map
Map.addLayer(ndvi, ndvi_vis_params, 'NDVI')

# Display the map
Map

Map(center=[28.602204452204358, 81.1268797516813], controls=(WidgetControl(options=['position', 'transparent_b…

In [6]:
# Function to calculate statistics
def calculate_stats(feature):
    # Calculate the pixel count and standard deviation of NDVI
    pixel_count = ndvi.reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=feature.geometry(),
        scale=10,
        maxPixels=1e13
    ).get('NDVI')

    # Return the feature with the calculated properties
    return feature.set({'pixel_count': pixel_count})

# Map the function over the FeatureCollection
kailali_stats_fc = kailali_fc.map(calculate_stats)

# Convert the FeatureCollection to a GeoDataFrame
kailali_stats_gdf = geemap.ee_to_gdf(kailali_stats_fc)

# Print the GeoDataFrame
print(kailali_stats_gdf)

                                              geometry  Unnamed_ 0  \
0    POLYGON ((81.13534 28.61148, 81.13546 28.61140...          56   
1    POLYGON ((81.13502 28.61497, 81.13572 28.61464...          57   
2    POLYGON ((81.13302 28.61406, 81.13310 28.61403...          58   
3    POLYGON ((81.13427 28.61318, 81.13443 28.61315...          59   
4    POLYGON ((81.13369 28.61117, 81.13385 28.61109...          60   
..                                                 ...         ...   
555  POLYGON ((81.16897 28.56115, 81.16928 28.56077...        1792   
556  POLYGON ((81.15392 28.56949, 81.15409 28.56889...        1800   
557  POLYGON ((81.15024 28.56951, 81.15045 28.56900...        1801   
558  POLYGON ((81.15108 28.56815, 81.15141 28.56792...        1802   
559  POLYGON ((81.14709 28.56167, 81.14783 28.56143...        1803   

                   _geolocati crop_monso crop_summe       crop_winte district  \
0    [28.6113178, 81.1355891]       rice     pulses           lentil  Kailali 

In [7]:
print('Min number of pixel within a polygon for kailali =', kailali_stats_gdf['pixel_count'].min())

Min number of pixel within a polygon for kailali = 5


#### Now move on to creating temporal profiling 

In [8]:
# print length of kailali district 
print('Total number of crops in kailali district :',len(kailali_stats_gdf))

Total number of crops in kailali district : 560


In [10]:
print(kailali_stats_gdf['crop_winte'].unique().tolist())

['lentil', 'wheat', nan, 'mustard', 'spices', 'lentil, mustard', 'sugarcane', 'grassland', 'mustard, lentil', 'vegetables', 'potato', 'shrub_tree', 'fallow', 'maize', 'spices, vegetables', 'banana', 'vegetables, vegetables', 'pea', 'flower', 'black_gram', 'spices, potato', 'black_gram, mustard']


In [11]:
# for the winter season we will create a temporal profile for each crop class's each polygon 
def mask_s2clouds(image):
    qa = image.select('QA60')

    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11

    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    return image.updateMask(mask).divide(10000).copyProperties(image, ['system:time_start'])

def add_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('ndvi')
    return image.addBands([ndvi])

def visualize_image(image):
    ndvi_vis = {
        'min': -0.2,
        'max': 0.8,
        'palette': ['#d73027','#f46d43','#fdae61','#fee08b',
                    '#ffffbf','#d9ef8b','#a6d96a','#66bd63','#1a9850']
    }
    ndvi_vis_image = image.visualize(**ndvi_vis).clip(geometry)
    return background.blend(ndvi_vis_image).selfMask()


In [12]:
dataset = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
    .filterDate('2022-01-01', '2024-01-01') \
    .map(mask_s2clouds) \
    .map(add_ndvi) \
    .filterBounds(aoi)


In [13]:
days = 15
millis = ee.Number(days).multiply(24 * 60 * 60 * 1000)

join = ee.Join.saveAll(matchesKey='images_joined')
diff_filter = ee.Filter.maxDifference(
    difference=millis,
    leftField='system:time_start',
    rightField='system:time_start'
)

joined_collection = join.apply(
    primary=dataset,
    secondary=dataset,
    condition=diff_filter
)

def compute_mean(image):
    matching_images = ee.ImageCollection.fromImages(image.get('images_joined'))
    mean_image = matching_images.reduce(
        ee.Reducer.mean().setOutputs(['moving_average'])
    )
    return ee.Image(image).addBands(mean_image)

smoothed_collection = ee.ImageCollection(
    joined_collection.map(compute_mean)
)


In [17]:
def export_ndvi_for_polygons(image_collection, feature_collection, output_folder):
    tasks = []
    for feature in feature_collection.getInfo()['features']:
        polygon = ee.Geometry.Polygon(feature['geometry']['coordinates'])
        out_dir = f"{output_folder}/{feature['properties']['Unnamed_ 0']}"
        task = ee.batch.Export.image.toDrive(
            image=image_collection.mean(),
            description=f"NDVI_{feature['properties']['Unnamed_ 0']}",
            folder=output_folder,
            fileNamePrefix=f"NDVI_{feature['properties']['Unnamed_ 0']}",
            scale=20,
            region=polygon
        )
        tasks.append(task)
        task.start()
    return tasks

# Assuming `feature_collection` is your collection of polygons
feature_collection = kailali_fc
export_tasks = export_ndvi_for_polygons(smoothed_collection.select('ndvi_moving_average'), feature_collection, 'export')


KeyboardInterrupt: 